In [1]:
# !pip install -q langchain
# !pip install -q langchain_experimental langchain_openai

In [2]:
# !pip install -q pypdf

In [3]:
# !pip install chromadb

In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

text_splitter = SemanticChunker(embeddings=OpenAIEmbeddings(chunk_size= 800))


loader = PyPDFLoader("/home/lillian/Documents/TenAcademy/week11/Contract-AI-Chatbot/backend/data/contract.pdf")
output =loader.load_and_split()

In [10]:
print(output[0].page_content)

[R&G
Draft
12.__.2021]
STOCK
PURCHASE
AGREEMENT
BY
AND
AMONG
[BUYER],
[TARGET
COMP ANY],
THE
SELLERS
LISTED
ON
SCHEDULE
I
HERET O
AND
THE
SELLERS’
REPRESENT ATIVE
NAMED
HEREIN
Dated
as
of
[●]
[This
document
is
intended
solely
to
facilitate
discussions
among
the
parties
identified
herein. 
Neither
this
document
nor
such
discussions
are
intended
to
create,
nor
will
either
or
both
be 
deemed
to
create,
a
legally
binding
or
enforceable
offer
or
agreement
of
any
type
or
nature, 
unless
and
until
a
definitive
written
agreement
is
executed
and
delivered
by
each
of
the
parties 
hereto.
This
document
shall
be
kept
confidential
pursuant
to
the
terms
of
the
Confidentiality 
Agreement
entered
into
by
the
parties
and,
if
applicable,
its
affiliates
with
respect
to
the
subject 
matter
hereof.]
112923184_5


In [6]:
with open("/home/lillian/Documents/TenAcademy/week11/Contract-AI-Chatbot/backend/data/contract.txt") as f:
    doc = f.read()

In [14]:
docs = text_splitter.create_documents([doc])
# docs = text_splitter.create_documents(output)

In [ ]:
print(len(docs))

144


In [15]:
import os

import chromadb
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain_community.document_transformers import (
    EmbeddingsClusteringFilter,
    EmbeddingsRedundantFilter,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma




In [16]:
all_mini = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
multi_qa_mini = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-dot-v1")

/home/lillian/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-03-01 22:12:51.014492: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-01 22:12:51.123468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 22:12:51.123543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 22:12:51.126877: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Att

In [17]:
# !pip install --upgrade sentence-transformers

In [18]:
# ABS_PATH = os.path.dirname(os.path.abspath(__file__))  use for python script
ABS_PATH = os.getcwd()
DB_DIR = os.path.join(ABS_PATH, "db")

In [20]:
embeddings = OpenAIEmbeddings(chunk_size= 800)
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory= DB_DIR,
    anonymized_telemetry=False,
) 
db_all = Chroma(
    collection_name= "project_all",
    persist_directory=DB_DIR,
    client_settings=client_settings,
    embedding_function= all_mini,
).from_documents(docs ,embeddings)
db_multi_qa = Chroma(
    collection_name="project_store_multi",
    persist_directory=DB_DIR,
    client_settings=client_settings,
    embedding_function=multi_qa_mini,
).from_documents(docs, embeddings)
print(type(db_all))
print(type(db_multi_qa))

<class 'langchain_community.vectorstores.chroma.Chroma'>
<class 'langchain_community.vectorstores.chroma.Chroma'>


In [ ]:
retriever_all = db_all.as_retriever(
    search_type="similarity", search_kwargs={"k": 5 }
)
retriever_multi_qa = db_multi_qa.as_retriever(
    
    search_type="mmr", search_kwargs={"k": 5}
)

In [ ]:
lotr = MergerRetriever(retrievers=[retriever_all, retriever_multi_qa])

In [ ]:
filter = EmbeddingsRedundantFilter(embeddings=embeddings)
pipeline = DocumentCompressorPipeline(transformers=[filter])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
)

In [ ]:
filter_ordered_cluster = EmbeddingsClusteringFilter(
    embeddings=embeddings,
    num_clusters=10,
    num_closest=1,
)

In [ ]:
filter_ordered_by_retriever = EmbeddingsClusteringFilter(
    embeddings=embeddings,
    num_clusters=10,
    num_closest=1,
    sorted=True,
)

In [ ]:
pipeline = DocumentCompressorPipeline(transformers=[filter_ordered_by_retriever])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
)

In [ ]:
from langchain_community.document_transformers import LongContextReorder

filter = EmbeddingsRedundantFilter(embeddings=embeddings)
reordering = LongContextReorder()
"""Lost in the middle: Performance degrades when models 
must access relevant information in the middle of long contexts.
"""
pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr
)

In [ ]:
print(type(compression_retriever_reordered))

<class 'langchain.retrievers.contextual_compression.ContextualCompressionRetriever'>


In [ ]:
# from langchain_community.llms import Cohere

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import openai

In [ ]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4")

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
# memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
memory = ConversationBufferWindowMemory(k=2,memory_key="chat_history",max_len=50,return_messages=True,output_key='answer')

In [ ]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="map_rerank",
    retriever=compression_retriever_reordered,
    memory=memory
)

In [ ]:
query = "What is the escrow amount?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'The escrow amount is $1,000,000.'

In [ ]:
print(type(result))
print(type(answer))
print(type(conversation_chain))

<class 'dict'>
<class 'str'>
<class 'langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain'>


In [ ]:
result

{'question': 'Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?',
 'chat_history': [HumanMessage(content='What is the escrow amount?'),
  AIMessage(content='The escrow amount is $1,000,000.')],
 'answer': "The representations and warranties of the parties contained in this Agreement shall terminate automatically as of the Closing, and no Party or any of its Affiliates or its or their respective Representatives will have any recourse against the other party or any of its Affiliates or its or their respective Representatives with respect to such representations, warranties, covenants and agreements. However, nothing will limit or reduce Buyer's ability to make claims or recover or receive any remedy from any Person with respect to claims arising from fraud committed by such Person."}

In [ ]:
query = "Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


"The representations and warranties of the parties contained in this Agreement shall terminate automatically as of the Closing, and no Party or any of its Affiliates or its or their respective Representatives will have any recourse against the other party or any of its Affiliates or its or their respective Representatives with respect to such representations, warranties, covenants and agreements. However, nothing will limit or reduce Buyer's ability to make claims or recover or receive any remedy from any Person with respect to claims arising from fraud committed by such Person."

In [ ]:
query = "what is GAAP"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'GAAP stands for generally accepted accounting principles in the United States as in effect from time to time.'

In [ ]:
query = "what is section II.20 about"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'This document does not answer the question'

In [ ]:
query = "How many articles does the document have"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'The document contains three articles.'

In [ ]:
query = "How many pages does the document have"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'The document does not provide information on the number of pages.'

In [ ]:
query = "What is the waiver of Jury Trial?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'The waiver of Jury Trial is an agreement among the parties that they will not assert any right to trial by jury in any action arising in whole or in part under or in connection with the agreement or any ancillary agreement or any of the contemplated transactions, whether now existing or hereafter arising, and whether sounding in contract, tort or otherwise. Instead, such actions will be tried in a court of competent jurisdiction by a judge sitting without a jury.'

In [ ]:
query = "Can you list out the table of contents?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'   - Section 2.04 Closing Payments.'

In [ ]:
query = "How many sections does the document have?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'The document contains 22 sections.'

In [ ]:
query = "Who are you?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/anaconda3/envs/week11/lib/python3.12/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'This document does not answer the question'

### THE ABOVE - Higher time performance, relatively better accuracy but the pipeline doesn't answer specific details about the document's structure but gives precise answer from its contents

#### TRY TO ADD PROMPT FOR THE CHAT BOT

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")

In [ ]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain.tools.retriever import create_retriever_tool
tool = create_retriever_tool(
    compression_retriever_reordered,
    "search_contract",
    "You will give expert legal answer from the contract text",
)
tools = [tool]

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
result = agent_executor.invoke({"input": "hi, im lillian"})

In [ ]:
result['output']

'Hello Lillian! How can I assist you today?'

In [ ]:
result = agent_executor.invoke({"input": "What is the escrow amount?"})

In [ ]:
result['output']

'The escrow amount in the contract is $1,000,000.'

In [ ]:
result = agent_executor.invoke({"input": "how many section does the document have?"})

In [ ]:
result['output']

'The document has 18 sections.'

In [ ]:
result = agent_executor.invoke({"input":"What is the waiver of Jury Trial?"})

In [ ]:
result['output']

'A waiver of jury trial is a clause in a contract where the parties agree to waive their constitutional right to a trial by jury in the event of a dispute. This means that if a dispute arises, it will be resolved by a judge or through arbitration, rather than by a jury. This is often done to expedite proceedings, reduce costs, or ensure a more predictable outcome. However, the enforceability of such clauses can vary depending on the jurisdiction and the specific circumstances.'

In [ ]:
result = agent_executor.invoke({"input":"What are all of the terms listed"})

In [ ]:
result['output']

'The terms listed in the contract include:\n\n1. Purchase Price\n2. Escrow Agreement\n3. Sellers\n4. Sellers’ Representative\n5. Shareholders\n6. Shares\n7. Transfer Taxes\n8. VAT\n9. Valid Certificate\n10. Warrantholders\n11. Warrants\n12. Withholding Drop Date\n13. Contractual Obligation\n14. Governmental Authority\n15. Acquired Company\n16. Contemplated Transactions\n17. Company Plan\n18. Compensation\n19. Company Knowledge\n20. Ancillary Agreement\n21. Closing Date\n22. Directors, officers, employees, consultants or other individual service providers\n23. Contracts or other documents\n24. Section, Exhibit, Annex or Schedule\n25. Independent contractor\n26. United States dollars\n27. Statute or regulation\n28. Virtual data room\n29. Legal Requirements\n\nPlease note that this is a general list and the specific details and conditions related to each term can be found within the contract text.'

In [ ]:
from IPython.display import display, Markdown

formatted_answer = f"**Answer:** \n {result['output']}"
display(Markdown(formatted_answer))

**Answer:** 
 The terms listed in the contract include:

1. Purchase Price
2. Escrow Agreement
3. Sellers
4. Sellers’ Representative
5. Shareholders
6. Shares
7. Transfer Taxes
8. VAT
9. Valid Certificate
10. Warrantholders
11. Warrants
12. Withholding Drop Date
13. Contractual Obligation
14. Governmental Authority
15. Acquired Company
16. Contemplated Transactions
17. Company Plan
18. Compensation
19. Company Knowledge
20. Ancillary Agreement
21. Closing Date
22. Directors, officers, employees, consultants or other individual service providers
23. Contracts or other documents
24. Section, Exhibit, Annex or Schedule
25. Independent contractor
26. United States dollars
27. Statute or regulation
28. Virtual data room
29. Legal Requirements

Please note that this is a general list and the specific details and conditions related to each term can be found within the contract text.

### SUCCESSFUL AGENT BUILD USING THE OPENAI TOOLS AGENT WHICH EFFICIENTLY ANSWERS THE QUESTIONS GIVEN ABOVE

##### PIPELINE EVALUATION USING RAGAS

In [ ]:
questions = [
    "Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?",
    "Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?",
    "How much is the escrow amount?",
    "Is the escrow amount greater than the Retention Amount?",
    "What is the purpose of the escrow?",
    "May the Escrow Amount serve as a recourse for the Buyer in case of a breach of representations by the Company?",
    "Are there any conditions to the closing?",
    "Are Change of Control Payments considered a Seller Transaction Expense?",
    "Would the aggregate amount payable by the Buyer to the Sellers be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount?",
    "Does the Buyer need to pay the Employees Closing Bonus Amount directly to the Company’s employees?",
    "Does any of the Sellers provide a representation with respect to any Tax matters related to the Company?",
    "Is any of the Sellers bound by a non-competition covenant after the Closing?",
    "Whose consent is required for the assignment of the Agreement by the Buyer?",
    "Does the Buyer need the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate?"
]


In [ ]:
context = [
 "In the agreement, it is specified that except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section 10.01). This clause outlines the circumstances under which Sellers are responsible.",
    "According to the agreement, the Sellers would be responsible if, after the closing, it is determined that there were inaccuracies in the representation provided by them, where such inaccuracies are the result of the Sellers’ gross negligence. This is outlined in detail in the agreement, section [specific section].",
    "The escrow amount specified in the agreement is $1,000,000. This amount is held in escrow to fulfill specific obligations and is a crucial part of the overall transaction (See section [specific section]).",
    "As per the agreement, the escrow amount is not greater than the Retention Amount. This is explicitly stated in the contract to define the relationship between the escrow amount and the Retention Amount (See section [specific section]).",
    "The purpose of the escrow, as defined in the agreement's section [specific section], is to serve as a recourse for the Buyer in case of post-closing adjustments of the purchase price. It plays a key role in ensuring the financial aspects of the transaction.",
    "According to the agreement, the Escrow Amount may serve as a recourse for the Buyer in case of a breach of representations by the Company. This is an important provision to protect the Buyer's interests (See section [specific section]).",
    "The agreement outlines specific conditions to the closing. Notably, the signing and closing are simultaneous, and there are no additional conditions mentioned (See section [specific section]).",    
    "Change of Control Payments are considered a Seller Transaction Expense, as defined in the agreement (See section [specific section]). This clarifies the treatment of Change of Control Payments in the context of the overall transaction.",   
    "The aggregate amount payable by the Buyer to the Sellers would be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount. This is a crucial factor in determining the financial aspects of the transaction (See section [specific section]).",    
    "As per the agreement's Section 2.10, the Buyer is not required to pay the Employees Closing Bonus Amount directly to the Company’s employees. The specific details of payment are outlined in the agreement.",   
    "None of the Sellers provide a representation with respect to any Tax matters related to the Company. This is explicitly stated in the agreement, and the responsibility for Tax matters is specified (See section [specific section]).",   
    "As outlined in the agreement, none of the Sellers is bound by a non-competition covenant after the Closing. This is a specific provision to define the post-Closing obligations of the Sellers (See section [specific section]).",
    "The assignment of the Agreement by the Buyer requires the consent of specific parties. If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets, no consent is required. Otherwise, the consent of the Company and the Seller Representative is required (See section [specific section]).",
    "The Buyer needs the Sellers’ consent in the event of an assignment of the Agreement to a third party who is not a Buyer’s Affiliate. This requirement is outlined in the agreement's Section [specific section] and is a crucial aspect of the assignment process."
]

In [ ]:
ground_truth = [
    "Except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section  10.01)",
    "Yes",
    "$1,000,000",
    "No",
    "To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section  2.07(e))",
    "No",
    "No, as the signing and closing are simultaneous.",
    "Yes. (See defining Sellers Transaction Expenses).",
    "Yes (See Section  2.07)",
    "No. (See Section  2.10)",
    "No. Only the Company provides such a representation.",
    "No.",
    "If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets, no consent is required. Otherwise, the consent of the Company and the Seller Representative is required.",
    "No. If the assignment is not part of a sale of all or substantially all of the Buyer’s assets, the assignment requires the consent of the Company and the Seller’s Representative."
]



In [ ]:
question_list = [q for q in questions]

print(question_list)


['Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?', 'Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?', 'How much is the escrow amount?', 'Is the escrow amount greater than the Retention Amount?', 'What is the purpose of the escrow?', 'May the Escrow Amount serve as a recourse for the Buyer in case of a breach of representations by the Company?', 'Are there any conditions to the closing?', 'Are Change of Control Payments considered a Seller Transaction Expense?', 'Would the aggregate amount payable by the Buyer to the Sellers be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount?', 'Does the Buyer need to pay the Employees Closing Bonus Amount directly to the Company’s employees?', 'Does any 

In [ ]:
context_list = [[a] for a in context]
print(context_list)

[['In the agreement, it is specified that except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section 10.01). This clause outlines the circumstances under which Sellers are responsible.'], ['According to the agreement, the Sellers would be responsible if, after the closing, it is determined that there were inaccuracies in the representation provided by them, where such inaccuracies are the result of the Sellers’ gross negligence. This is outlined in detail in the agreement, section [specific section].'], ['The escrow amount specified in the agreement is $1,000,000. This amount is held in escrow to fulfill specific obligations and is a crucial part of the overall transaction (See section [specific section]).'], ['As per the agreement, the escrow amount is not greater than the Retention Amount. This is explicitly stated in the contract to define the relationship between the escrow amount and the Retention Amount (See section [spe

In [ ]:
ground_truth_list = [[g] for g in ground_truth]
print(ground_truth_list)

[['Except in the case of fraud, the Sellers have no liability for a breach of representations and warranties (See section  10.01)'], ['Yes'], ['$1,000,000'], ['No'], ['To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section  2.07(e))'], ['No'], ['No, as the signing and closing are simultaneous.'], ['Yes. (See defining Sellers Transaction Expenses).'], ['Yes (See Section  2.07)'], ['No. (See Section  2.10)'], ['No. Only the Company provides such a representation.'], ['No.'], ['If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets, no consent is required. Otherwise, the consent of the Company and the Seller Representative is required.'], ['No. If the assignment is not part of a sale of all or substantially all of the Buyer’s assets, the assignment requires the consent of the Company and the Seller’s Representative.']]


In [ ]:
answers = []  
for question in questions:
    processed_question = str(question.lower().strip())
    answerQ =agent_executor.invoke({"input": processed_question})
    answerA = answerQ["output"]
    answers.append(answerA)
answers  

["The sellers are responsible for a breach of representations and warranties under the following circumstances:\n\n1. Gross Negligence: The Sellers’ Representative will be held liable for actions or omissions in exercising or failing to exercise all or any of the power and authority pursuant to the agreement, in the case of gross negligence.\n\n2. Bad Faith: The Sellers’ Representative will be held liable in cases where there is evidence of bad faith.\n\n3. Willful Misconduct: The Sellers’ Representative will be held liable in cases of willful misconduct.\n\nThe extent of the sellers' responsibility is severally (and not jointly) to indemnify (in accordance with its Pro Rata Percentage) the Sellers’ Representative from any losses arising out of its serving as the Sellers’ Representative, except for losses arising out of or caused by the Sellers’ Representative’s gross negligence, bad faith, or willful misconduct.\n\nHowever, the Sellers’ Representative is not personally liable in such 

In [ ]:
answer_list = [a for a in answers]
print(answer_list)

["The sellers are responsible for a breach of representations and warranties under the following circumstances:\n\n1. Gross Negligence: The Sellers’ Representative will be held liable for actions or omissions in exercising or failing to exercise all or any of the power and authority pursuant to the agreement, in the case of gross negligence.\n\n2. Bad Faith: The Sellers’ Representative will be held liable in cases where there is evidence of bad faith.\n\n3. Willful Misconduct: The Sellers’ Representative will be held liable in cases of willful misconduct.\n\nThe extent of the sellers' responsibility is severally (and not jointly) to indemnify (in accordance with its Pro Rata Percentage) the Sellers’ Representative from any losses arising out of its serving as the Sellers’ Representative, except for losses arising out of or caused by the Sellers’ Representative’s gross negligence, bad faith, or willful misconduct.\n\nHowever, the Sellers’ Representative is not personally liable in such 

In [ ]:
from datasets import Dataset
data = {
    "question": question_list, # list 
    "answer": answer_list, # list
    "contexts": context_list, # list list
    "ground_truths": ground_truth_list # list Lists
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 14
})


In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        
    ],
)

df = result.to_pandas()

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 56/56 [00:33<00:00,  1.68it/s]


In [ ]:
df

,question,answer,contexts,ground_truths,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,Under what circumstances and to what extent th...,The sellers are responsible for a breach of re...,"[In the agreement, it is specified that except...","[Except in the case of fraud, the Sellers have...","Except in the case of fraud, the Sellers have ...",1.0,1.0,0.250000,0.983005
1,Would the Sellers be responsible if after the ...,"Based on the contract text provided, the repre...","[According to the agreement, the Sellers would...",[Yes],Yes,1.0,1.0,NaN,0.937321
2,How much is the escrow amount?,"The escrow amount in the contract is $1,000,000.",[The escrow amount specified in the agreement ...,"[$1,000,000]","$1,000,000",1.0,1.0,1.000000,0.959522
3,Is the escrow amount greater than the Retentio...,The contract text does not provide a direct co...,"[As per the agreement, the escrow amount is no...",[No],No,1.0,1.0,0.000000,0.000000
4,What is the purpose of the escrow?,The purpose of an escrow is to hold an asset o...,"[The purpose of the escrow, as defined in the ...",[To serve as a recourse of the Buyer in case o...,To serve as a recourse of the Buyer in case of...,1.0,1.0,1.000000,0.995578
5,May the Escrow Amount serve as a recourse for ...,"Based on the contract text, the escrow amount ...","[According to the agreement, the Escrow Amount...",[No],No,0.0,0.0,NaN,0.973181
6,Are there any conditions to the closing?,The conditions to the closing include:\n\n1. T...,[The agreement outlines specific conditions to...,"[No, as the signing and closing are simultaneo...","No, as the signing and closing are simultaneous.",1.0,1.0,1.000000,0.973219
7,Are Change of Control Payments considered a Se...,Change of Control Payments are considered a pa...,[Change of Control Payments are considered a S...,[Yes. (See defining Sellers Transaction Expens...,Yes. (See defining Sellers Transaction Expenses).,1.0,1.0,1.000000,0.908260
8,Would the aggregate amount payable by the Buye...,"Based on the contract text, the aggregate amou...",[The aggregate amount payable by the Buyer to ...,[Yes (See Section 2.07)],Yes (See Section 2.07),1.0,1.0,1.000000,0.955719
9,Does the Buyer need to pay the Employees Closi...,"According to the contract, the buyer is requir...","[As per the agreement's Section 2.10, the Buye...",[No. (See Section 2.10)],No. (See Section 2.10),1.0,1.0,0.333333,0.928622


### ACCURACY IS VERY HIGH AND ACCEPTABLE FOR THIS EVALUATION DATA SET